In [10]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn import svm
from copy import deepcopy

/home/liuxiao/anaconda3/envs/torchenv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [20]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_word2vec','h_gpt2','h_gpt3emb','h_gpt3promptcurie','h_gpt3promptdav']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print('[Worker ID]',json.loads(data[-1])['workerId'])
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine','f':'Machine','g':'Catch'}[pagename.split('_')[2]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'','b':'word2vec','c':'gpt2','d':'gpt3emb','e':'gpt3promptcurie','f':'gpt3promptdav'}[pagename.split('_')[2]]

def get_imbedding_from_pagename(pagename,d,response_path='/home/liuxiao/TuringGithubRunnable/wordAssociation/Feedback_Word/static/responses_wordassoc'):
    html = pagename.split('/')[-1]
    with open(response_path+'/'+html,'r') as f:
        text = f.readlines()[0]
    response = text.split('Association: ')[-1].split('</p>')[0].strip().lower()
    cueWord = text.split('; Association: ')[0].split('Cue: ')[-1].strip().lower()
    response_emb = np.array(d[response])
    cueWord_emb = np.array(d[cueWord])
    return (response_emb-cueWord_emb)**2

def get_sample_from_pagename(pagename,d):
    x = get_imbedding_from_pagename(pagename,d)
    y = {'Human':0,'Machine':1}[get_gt_from_pagename(pagename)]
    return x,y
    
with open("/home/liuxiao/TuringGithub/XiaoData/word_openai_embedding_dict.json",'r') as f:
    word_openai_embedding_dict = json.load(f)

with open("/home/liuxiao/TuringGithub/XiaoData/word_word2vec_embedding_dict.json",'r') as f:
    word_word2vec_embedding_dict = json.load(f)  

word_embedding_dict = word_word2vec_embedding_dict
    
conn = sqlite3.connect("ec2_db/turing_feedback_word.db")
cursor = conn.cursor()
cursor.execute("select * from TuringAssociationAI_turing;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'word2vec':1,'gpt2':1,'gpt3emb':1,'gpt3promptcurie':1,'gpt3promptdav':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'word2vec':'Word2Vec','gpt2':'GPT2','gpt3emb':'GPT3(Emb)','gpt3promptcurie':'GPT3(pr, Curie)','gpt3promptdav':'GPT3(pr, DaVinci)'}

count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
confmat_overall_AI = construct_confmat()
ai_judge = {}
trialnum = 0
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        time_used, counterbalance = print_metadata(all_data[i])
        completed_counterbalances.append(counterbalance)
        marks = []
        marks_conf = []
        confmatrices = construct_confmat() # without catch
        confmat_AI = construct_confmat()

        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
#         confmatrices_without_catch = construct_confmat()
#     if True:
        
        participant_count += 1
        nTrial = 0
        X_train, y_train, X_test, y_test = [], [], [], []
        test_samples = []
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']

                
#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']
                
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 40:  
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))
                    
                    ### === Add for AI === ### 
                    x,y = get_sample_from_pagename(trial_data['trialData']['page_name'],word_embedding_dict)
                    X_test.append(x)
                    y_test.append(y)
                    trial_data['trialData']["machine_groundtruth"] = get_model_from_pagename(trial_data['trialData']['page_name'])
                    test_samples.append(deepcopy(trial_data['trialData']))
                    
                elif nTrial <= 40:
                    ### === Add for AI === ###
                    x,y = get_sample_from_pagename(trial_data['trialData']['page_name'],word_embedding_dict)
                    X_train.append(x)
                    y_train.append(y)
                    
#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=40:
                    continue
                
                count += 1
        print('Human Judge',confmatrices)
        print('catch',catch_trial_mat)                
        ### ===================== AI judge ======================== ###
#         assert len(X_train) == 40 and len(y_train) == 40 and len(X_test) == 40 and len(y_test) == 40
        clf = svm.SVC(verbose=True)
        clf.fit(list(X_train), y_train)
        predictions = clf.predict(X_test)
        pred_indx = 0
                
        for sample in test_samples:
            
            trialnum+=1
            response_speaker = ''
            if(predictions[pred_indx]==0):
                response_speaker = "human"
            elif(predictions[pred_indx]==1):
                response_speaker = "machine"

            confmat_AI['h_machine'][label_dict[sample['correct response']]][predictions[pred_indx]] += 1
            confmat_overall_AI['h_machine'][label_dict[sample['correct response']]][predictions[pred_indx]] += 1

            if sample['correct response'].lower() == 'machine':  
                model = sample["machine_groundtruth"]
                confmat_AI['h_{}'.format(model)][0][predictions[pred_indx]] += 1
                confmat_overall_AI['h_{}'.format(model)][0][predictions[pred_indx]] += 1

            trialdata = {"response_speaker": response_speaker, "groundtruth":sample["correct response"].lower(), "machine_groundtruth":sample["machine_groundtruth"]}
            ai_judge['Trial_Word_'+str(trialnum)] = {"trialData": trialdata}
            pred_indx+=1  

#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print('AI Judge',confmat_AI)
conn.close()

[Worker ID] A38DC3BG1ZCVZ2
[Finished Date] 2025-04-03 07:53:39
[Time Used] 7.2 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '8', 'difficulty': '9', 'mode': 'live', 'gender': 'female', 'education': 'bachelor', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
Human Judge {'h_machine': array([[14.,  6.],
       [10., 10.]]), 'h_word2vec': array([[0., 4.]]), 'h_gpt2': array([[2., 2.]]), 'h_gpt3emb': array([[4., 0.]]), 'h_gpt3promptcurie': array([[2., 2.]]), 'h_gpt3promptdav': array([[2., 2.]])}
catch [[ 0. 10.]]
[LibSVM].*
optimization finished, #iter = 58
obj = -23.188690, rho = -0.652939
nSV = 38, nBSV = 21
Total nSV = 38
AI Judge {'h_machine': array([[10., 10.],
       [ 5., 15.]]), 'h_word2vec': array([[0., 4.]]), 'h_gpt2': array([[1., 3.]]), 'h_gpt3emb': array([[1., 3.]]), 'h_gpt3promptcurie': array([[3., 1.]]), 'h_gpt3promptdav': array([[0., 4.]])}
[Worker ID] A3UBB1LJKDQGZP
[Finished Date] 2025-04-03 07:54:56
[Time Used] 7.

[LibSVM]*.*
optimization finished, #iter = 42
obj = -24.533935, rho = -0.172080
nSV = 39, nBSV = 25
Total nSV = 39
AI Judge {'h_machine': array([[13.,  7.],
       [ 7., 13.]]), 'h_word2vec': array([[0., 4.]]), 'h_gpt2': array([[0., 4.]]), 'h_gpt3emb': array([[2., 2.]]), 'h_gpt3promptcurie': array([[4., 0.]]), 'h_gpt3promptdav': array([[1., 3.]])}
[Worker ID] A31XT6RPLN6359
[Finished Date] 2025-04-03 12:23:17
[Time Used] 13.7 mins
[Questionnaire] {'country': 'United States', 'age': '45-50', 'engagement': '8', 'difficulty': '10', 'mode': 'live', 'gender': 'female', 'education': 'high school', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
Human Judge {'h_machine': array([[13.,  7.],
       [10., 10.]]), 'h_word2vec': array([[1., 3.]]), 'h_gpt2': array([[2., 2.]]), 'h_gpt3emb': array([[3., 1.]]), 'h_gpt3promptcurie': array([[2., 2.]]), 'h_gpt3promptdav': array([[2., 2.]])}
catch [[4. 6.]]
[LibSVM]*.*
optimization finished, #iter = 43
obj = -24.433132, rho = -0.

In [14]:
confmat_overall

{'h_machine': array([[661., 339.],
        [561., 439.]]),
 'h_word2vec': array([[ 69., 131.]]),
 'h_gpt2': array([[116.,  84.]]),
 'h_gpt3emb': array([[129.,  71.]]),
 'h_gpt3promptcurie': array([[138.,  62.]]),
 'h_gpt3promptdav': array([[109.,  91.]])}

In [22]:
confmat_overall_AI

{'h_machine': array([[616., 384.],
        [287., 713.]]),
 'h_word2vec': array([[  2., 198.]]),
 'h_gpt2': array([[ 49., 151.]]),
 'h_gpt3emb': array([[ 72., 128.]]),
 'h_gpt3promptcurie': array([[103.,  97.]]),
 'h_gpt3promptdav': array([[ 61., 139.]])}

In [23]:
with open("/home/liuxiao/TuringGithubRunnable/wordAssociation/Plot/Data_all/AIjudge_word2vec_Word_Feedback_testphase.json", 'w') as f:
    json.dump(ai_judge,f)

In [24]:
with open("/home/liuxiao/TuringGithubRunnable/wordAssociation/Plot/Data_all/AIjudge_openai_Word_Feedback_testphase.json", 'r') as f:
    open_ai_judge =  json.load(f)
word2vec_judge = ai_judge

In [25]:
ai_judge = {}
trialnum = 1
for k,v in open_ai_judge.items():
    ai_judge['Trial_Word_'+str(trialnum)] = v
    trialnum += 1

for k,v in word2vec_judge.items():
    ai_judge['Trial_Word_'+str(trialnum)] = v
    trialnum += 1

In [27]:
with open("/home/liuxiao/TuringGithubRunnable/wordAssociation/Plot/Data_all/AIjudge_compiled_Word_Feedback_testphase.json", 'w') as f:
    json.dump(ai_judge,f)